In [1]:
# Version to run in console

files_skipped <- 0
files_not_skipped <- 0

library(CpGWAS)
library(data.table)
setDTthreads(6)

library(stringr)
library(optparse)
library(doParallel)
library(foreach)

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [2]:
# library(future.apply)
# plan(multicore)

Loading required package: future



In [11]:
cl <- makeCluster(6)  # Leave one core free for other tasks
registerDoParallel(cl)

In [4]:
chr_list <- 1:22

this_chr <- 9

machine <- "notlocal"

print(paste0("chr: ", this_chr))
opt <- list(genome_file_index = this_chr,
            data_file = "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts/12-OUT_matched_SNP_meth_cov_outputs.csv",
            cores = 1)

#if(opt$cores > 1) registerDoParallel(opt$cores)

# Load genome files
genome_files <- list.files("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas_flipped",
                           pattern = "EUR", full.names = TRUE)
genome_files <- genome_files[grepl("allele", genome_files)]
genome_files <- genome_files[grepl("a2", genome_files)]

genome_files <- genome_files[grepl("pvar", genome_files)]

genome_files <- data.table(path = genome_files, Chr = NA)

genome_files$Chr <- str_split_fixed(genome_files$path, "chr", 2)[, 2]
genome_files$Chr <- gsub("\\.pvar", "", genome_files$Chr)

genome_files$Chr <- as.integer(genome_files$Chr)
genome_files <- genome_files[order(genome_files$Chr), ]

df <- fread(opt$data_file)

if(machine == "local"){
  df$path <- gsub("..//output_EXPANSE_a2", "~/data/stage1_output_a3/output_EXPANSE_dt", df$path)
}


df$path <- gsub("_a2_", "_dt_", df$path)
df$path <- gsub("\\.rds", "_dt_combined.csv", df$path)

print("Starting genome file processing")
# Process the specified genome file
g <- opt$genome_file_index
print(paste("Processing genome file index:", g))

paths <- list(
  pvar_path = genome_files[g]$path,
  pgen_path = gsub("pvar", "pgen", genome_files[g]$path),
  psam_path = gsub("pvar", "psam", genome_files[g]$path)
)

chr <- as.numeric(gsub("\\.pvar", "",
                       stringr::str_split_fixed(paths$pvar_path, "chr", 2)[, 2]))

my_SNPs <- CpGWAS::loadSNPData(paths$pvar_path, paths$pgen_path, paths$psam_path)
setkey(my_SNPs$pvar_dt, POS)
df_this_chr <- df[which(df$Chr == genome_files[g]$Chr), ]


summary_stats_list <- list.files("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas", pattern = "alleleprocessed", full.names = TRUE)
summary_stats_list <- summary_stats_list[grepl("a3", summary_stats_list)]

# Pre-load all summary stats files into a list and clean/standardize column names
summary_stats_data <- lapply(summary_stats_list, function(path) {
  cols_to_load <- c("CHR", "BP", "SNP", "A1", "A2", "Z", "SE", "PVAL")
  stats <- suppressWarnings(data.table::fread(path, select = cols_to_load))
  
  # Rename column if needed (uncomment if column name needs changing from "#CHROM" to "CHR")
  # setnames(stats, "#CHROM", "CHR")
  
  # Set key for efficient subsetting
  setkey(stats, BP)
  
  # Subset to only include rows where 'CHR' matches the 'chr' variable
  stats <- stats[CHR == chr]
  
  # Clean and standardize column names if needed
  # clean_and_standardize_colnames(stats)
  
  return(stats)
})

summary_stats_data <- lapply(summary_stats_data, function(stats) stats[`CHR` == genome_files[g]$Chr])

print("Loaded SNP data")
print("Files for this Chr:")
print(nrow(df_this_chr))

[1] "chr: 9"
[1] "Starting genome file processing"
[1] "Processing genome file index: 9"
[1] "Loaded SNP data"
[1] "Files for this Chr:"
[1] 486


In [5]:
j <- 1

print(paste0("File number: ", j))

start_time <- Sys.time()  # Start timing the loop iteration

if (grepl("empty", df_this_chr$path[j])) {
message(paste0("No model for ", df_this_chr$path[j]))
next
}

all_files_exist <- TRUE
outnames <- vector("character", length(summary_stats_list))

# Check if output files already exist
for (k in 1:length(summary_stats_list)) {
outnames[k] <- gsub("\\.rds$", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
if (!file.exists(outnames[k])) {
  all_files_exist <- FALSE
  break  # Exit the loop early if any file does not exist
}
}

# Load the CSV file
load_csv_start <- Sys.time()
my_csv <- fread(df_this_chr$path[j])
cgs <- levels(factor(my_csv$cg))
cgs <- as.integer(cgs)

setkey(my_csv,
     #cg,
     features)

my_csv_list <- split(my_csv, by = "cg")

# Ensure that 'cgs' is ordered and unique if it's not already
cgs <- unique(cgs[order(cgs)])

#print(paste("Loaded csv file:", df_this_chr$path[j]))
load_csv_end <- Sys.time()
#print(paste("Time to load CSV:", load_csv_end - load_csv_start))

[1] "File number: 1"


In [6]:
k <- 1
process_start <- Sys.time()

outname <- gsub("_dt_combined.csv", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])

summary_stats <- summary_stats_data[[k]]

matching_bps <- unique(my_csv$features)

# Subset summary_stats to only include rows with BPs in the matching_bps vector
summary_stats_sub <- summary_stats[BP %in% matching_bps]

MWASmodels <- vector("list", length(cgs))


In [50]:
Sys.time()

[1] "2024-08-29 06:37:49 PDT"

In [51]:
library(parallel)

In [52]:
# Using lapply with indices passed properly using a wrapper list
MWASmodels <- mclapply(function(this_cg_data) {
  if (!is.null(this_cg_data)) {
    model_result <- process_model_csv(this_cg_data, my_SNPs, summary_stats_sub)
    return(model_result)
  } else {
    return(NULL)  # Return NULL for null data to maintain index alignment
  }
}, my_csv_list, SIMPLIFY = FALSE)

Warning message in mclapply(function(this_cg_data) {:
“all scheduled cores encountered errors in user code”


In [53]:
Sys.time()

[1] "2024-08-29 06:37:50 PDT"

In [43]:
cl <- parallel::makeCluster(2)
parallel::clusterExport(cl, c("paths", "summary_stats_sub", "process_model_csv"))
parallel::clusterEvalQ(cl, {
  library(data.table)
  library(CpGWAS)
})

MWASmodels <- parallel::parLapply(cl, my_csv_list, function(this_cg_data) {
  my_SNPs <- CpGWAS::loadSNPData(paths$pvar_path, paths$pgen_path, paths$psam_path)
  setkey(my_SNPs$pvar_dt, POS)

  if (!is.null(this_cg_data)) {
    return(process_model_csv(this_cg_data, my_SNPs, summary_stats_sub))
  } else {
    return(NULL)
  }
})

parallel::stopCluster(cl)

[[1]]
[1] "CpGWAS"     "data.table" "stats"      "graphics"   "grDevices" 
[6] "utils"      "datasets"   "methods"    "base"      

[[2]]
[1] "CpGWAS"     "data.table" "stats"      "graphics"   "grDevices" 
[6] "utils"      "datasets"   "methods"    "base"

In [ ]:
Sys.time()

In [33]:
MWASmodels <- parallel::mclapply(my_csv_list, function(this_cg_data) {
  if (!is.null(this_cg_data)) {
    return(process_model_csv(this_cg_data, my_SNPs, summary_stats_sub))
  } else {
    return(NULL)
  }
}, mc.cores = 2)

Warning message in parallel::mclapply(my_csv_list, function(this_cg_data) {:
“scheduled core 2 encountered error in user code, all values of the job will be affected”


In [34]:
Sys.time()

[1] "2024-08-29 06:29:14 PDT"

In [28]:
head(MWASmodels)

$`172730`
        z         p         n 
0.9762807 0.3289254 1.0000000 

$`172895`
         z          p          n 
-0.9393989  0.3475260  2.0000000 

$`173292`
        z         p         n 
0.9762807 0.3289254 1.0000000 

$`173359`
         z          p          n 
-0.9371034  0.3487054  2.0000000 

$`173509`
        z         p         n 
0.9762807 0.3289254 1.0000000 

$`173918`
        z         p         n 
0.9762807 0.3289254 1.0000000

In [17]:
Sys.time()

[1] "2024-08-29 06:25:15 PDT"

In [18]:
library(parallel)

MWASmodels <- mclapply(seq_along(my_csv_list), function(idx) {
  this_cg_data <- my_csv_list[[idx]]
  if (!is.null(this_cg_data)) {
    model_result <- process_model_csv(this_cg_data, my_SNPs, summary_stats_sub)
    model_result['bp'] <- cgs[idx]
    return(model_result)
  } else {
    return(NULL)
  }
}, mc.cores = 2)

MWASmodels <- Filter(Negate(is.null), MWASmodels)

Warning message in mclapply(seq_along(my_csv_list), function(idx) {:
“scheduled core 2 encountered error in user code, all values of the job will be affected”


In [19]:
Sys.time()

[1] "2024-08-29 06:25:24 PDT"

In [9]:
# # Using lapply with indices passed properly using a wrapper list
# MWASmodels <- mapply(function(this_cg_data, idx) {
#   if (!is.null(this_cg_data)) {
#     model_result <- process_model_csv(this_cg_data, my_SNPs, summary_stats_sub)
#     model_result['bp'] <- cgs[idx]  # Assign the correct CG identifier
#     return(model_result)
#   } else {
#     return(NULL)  # Return NULL for null data to maintain index alignment
#   }
# }, my_csv_list, seq_along(my_csv_list), SIMPLIFY = FALSE)

In [10]:
Sys.time()

[1] "2024-08-29 06:23:22 PDT"

In [9]:
# # Now, use this pre-structured data in your loop to eliminate the bottleneck
# for (i in seq_len(length(cgs))) {
#   print(i)
#   print(Sys.time())
#   this_cg <- cgs[i]
#   this_cg_data <- my_csv_list[[as.character(this_cg)]]
#   if (!is.null(this_cg_data)) {
#     MWASmodels[[i]] <- process_model_csv(this_cg_data, my_SNPs, summary_stats_sub)
#     MWASmodels[[i]]['bp'] <- MWASmodels[[i]]['bp'] <- this_cg
#   }
# }

In [10]:
Sys.time()

[1] "2024-08-29 06:14:39 PDT"

In [11]:
head(MWASmodels)

$`172730`
           z            p            n           bp 
9.762807e-01 3.289254e-01 1.000000e+00 1.727300e+05 

$`172895`
            z             p             n            bp 
-9.393989e-01  3.475260e-01  2.000000e+00  1.728950e+05 

$`173292`
           z            p            n           bp 
9.762807e-01 3.289254e-01 1.000000e+00 1.732920e+05 

$`173359`
            z             p             n            bp 
-9.371034e-01  3.487054e-01  2.000000e+00  1.733590e+05 

$`173509`
           z            p            n           bp 
9.762807e-01 3.289254e-01 1.000000e+00 1.735090e+05 

$`173918`
           z            p            n           bp 
9.762807e-01 3.289254e-01 1.000000e+00 1.739180e+05

In [12]:
Sys.time()

[1] "2024-08-29 06:14:47 PDT"

In [16]:
library(future.apply)
plan(multisession)  # or multicore if on Unix-like systems for testing

# Use options to capture and debug future errors
options(future.globals.maxSize = Inf)  # Adjust if global size is an issue
future::future_options(future.globals.onReference = "error")

MWASmodels2 <- future_mapply(function(this_cg_data, idx) {
    tryCatch({
        if (!is.null(this_cg_data)) {
            model_result <- process_model_csv(this_cg_data, my_SNPs, summary_stats_sub)
            model_result['bp'] <- cgs[idx]  # Assign the correct CG identifier
            return(model_result)
        } else {
            return(NULL)  # Handle NULL data
        }
    }, error = function(e) {
        message("Error in processing: ", e$message)  # Print error message
        return(NULL)  # Return NULL on error to maintain list structure
    })
}, my_csv_list, seq_along(my_csv_list), SIMPLIFY = FALSE, future.seed = TRUE)

ERROR: Error: 'future_options' is not an exported object from 'namespace:future'


In [15]:
MWASmodels2 <- future_mapply(function(this_cg_data, idx) {
  if (!is.null(this_cg_data)) {
    model_result <- process_model_csv(this_cg_data, my_SNPs, summary_stats_sub)
    model_result['bp'] <- cgs[idx]  # Assign the correct CG identifier
    return(model_result)
  } else {
    return(NULL)  # Return NULL for null data to maintain index alignment
  }
}, my_csv_list, seq_along(my_csv_list), SIMPLIFY = FALSE, future.seed = TRUE)

ERROR: Error in eval(expr, envir, enclos): PgrGetD() error 6


In [ ]:
Sys.time()

In [21]:
MWASmodels <- Filter(Negate(is.null), MWASmodels)  # Remove NULL entries
results_end <- Sys.time()
# print(paste("Time to create MWAS results:", results_end - results_start))

# Save the results
save_start <- Sys.time()
#saveRDS(results, outname)

df <- do.call(rbind, MWASmodels)
df <- as.data.frame(df)

outname_csv <- gsub("rds", "csv", outname)
fwrite(df, outname_csv)
save_end <- Sys.time()
#print(paste("Saved results to:", outname))
#print(paste("Time to save results:", save_end - save_start))

process_end <- Sys.time()

In [22]:
head(df)

,z,p,n,bp
,<dbl>,<dbl>,<dbl>,<dbl>
172730,0.9762807,0.3289254,1,172730
172895,-0.9393989,0.3475260,2,172895
173292,0.9762807,0.3289254,1,173292
173359,-0.9371034,0.3487054,2,173359
173509,0.9762807,0.3289254,1,173509
173918,0.9762807,0.3289254,1,173918


In [9]:
for(j in 1:nrow(df_this_chr)){
#for(j in 1:3){
  print(paste0("File number: ", j))
  
  start_time <- Sys.time()  # Start timing the loop iteration
  
  if (grepl("empty", df_this_chr$path[j])) {
    message(paste0("No model for ", df_this_chr$path[j]))
    next
  }
  
  all_files_exist <- TRUE
  outnames <- vector("character", length(summary_stats_list))
  
  # Check if output files already exist
  for (k in 1:length(summary_stats_list)) {
    outnames[k] <- gsub("\\.rds$", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
    if (!file.exists(outnames[k])) {
      all_files_exist <- FALSE
      break  # Exit the loop early if any file does not exist
    }
  }
  
  # Load the CSV file
  load_csv_start <- Sys.time()
  my_csv <- fread(df_this_chr$path[j])
  cgs <- levels(factor(my_csv$cg))
  cgs <- as.integer(cgs)
  
  setkey(my_csv,
         #cg,
         features)
  
  my_csv_list <- split(my_csv, by = "cg")
  
  # Ensure that 'cgs' is ordered and unique if it's not already
  cgs <- unique(cgs[order(cgs)])
  
  #print(paste("Loaded csv file:", df_this_chr$path[j]))
  load_csv_end <- Sys.time()
  #print(paste("Time to load CSV:", load_csv_end - load_csv_start))
  
  # Process each summary statistics file
  for (k in 1:length(summary_stats_list)) {
    process_start <- Sys.time()
    
    outname <- gsub("_dt_combined.csv", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])

    summary_stats <- summary_stats_data[[k]]
    
    matching_bps <- unique(my_csv$features)

    # Subset summary_stats to only include rows with BPs in the matching_bps vector
    summary_stats_sub <- summary_stats[BP %in% matching_bps]
    
    MWASmodels <- vector("list", length(cgs))
    
    
    # Now, use this pre-structured data in your loop to eliminate the bottleneck
    for (i in seq_len(length(cgs))) {
      print(i)
      print(Sys.time())
      this_cg <- cgs[i]
      this_cg_data <- my_csv_list[[as.character(this_cg)]]
      if (!is.null(this_cg_data)) {
        MWASmodels[[i]] <- process_model_csv(this_cg_data, my_SNPs, summary_stats_sub)
        MWASmodels[[i]]['bp'] <- MWASmodels[[i]]['bp'] <- this_cg
      }
    }
  
    
    MWASmodels <- Filter(Negate(is.null), MWASmodels)  # Remove NULL entries
    results_end <- Sys.time()
    # print(paste("Time to create MWAS results:", results_end - results_start))
    
    # Save the results
    save_start <- Sys.time()
    #saveRDS(results, outname)
    
    df <- do.call(rbind, MWASmodels)
    df <- as.data.frame(df)
    
    outname_csv <- gsub("rds", "csv", outname)
    fwrite(df, outname_csv)
    save_end <- Sys.time()
    #print(paste("Saved results to:", outname))
    #print(paste("Time to save results:", save_end - save_start))
    
    process_end <- Sys.time()
    #print(paste("Total time to process summary stats file", k, ":", process_end - process_start))
  }
  
  loop_end_time <- Sys.time()  # End timing the loop iteration
  print(paste("Total time for file number", j, ":", loop_end_time - start_time))
}

In [4]:
summary_stats_list

character(0)

In [ ]:
files_skipped <- 0
files_not_skipped <- 0

library(CpGWAS)
library(data.table)
library(stringr)
library(optparse)
library(doParallel)
library(foreach)

chr_list <- 1:22

for(this_chr in chr_list){
print(paste0("chr: ", this_chr))
opt <- list(genome_file_index = this_chr,
            data_file = "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts/12-OUT_matched_SNP_meth_cov_outputs.csv",
            cores = 1)

#if(opt$cores > 1) registerDoParallel(opt$cores)

# Load genome files
genome_files <- list.files("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas_flipped",
                           pattern = "EUR", full.names = TRUE)
genome_files <- genome_files[grepl("allele", genome_files)]
genome_files <- genome_files[grepl("a2", genome_files)]

genome_files <- genome_files[grepl("pvar", genome_files)]

genome_files <- data.table(path = genome_files, Chr = NA)

genome_files$Chr <- str_split_fixed(genome_files$path, "chr", 2)[, 2]
genome_files$Chr <- gsub("\\.pvar", "", genome_files$Chr)

genome_files$Chr <- as.integer(genome_files$Chr)
genome_files <- genome_files[order(genome_files$Chr), ]

df <- fread(opt$data_file)
df$path <- gsub("_a2_", "_dt_", df$path)
df$path <- gsub("\\.rds", "_dt_combined.csv", df$path)

print("Starting genome file processing")
# Process the specified genome file
g <- opt$genome_file_index
print(paste("Processing genome file index:", g))

paths <- list(
  pvar_path = genome_files[g]$path,
  pgen_path = gsub("pvar", "pgen", genome_files[g]$path),
  psam_path = gsub("pvar", "psam", genome_files[g]$path)
)

chr <- as.numeric(gsub("\\.pvar", "",
            stringr::str_split_fixed(paths$pvar_path, "chr", 2)[, 2]))

my_SNPs <- CpGWAS::loadSNPData(paths$pvar_path, paths$pgen_path, paths$psam_path)
setkey(my_SNPs$pvar_dt, `#CHROM`, POS)
df_this_chr <- df[which(df$Chr == genome_files[g]$Chr), ]


summary_stats_list <- list.files("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas", pattern = "alleleprocessed", full.names = TRUE)
summary_stats_list <- summary_stats_list[grepl("a3", summary_stats_list)]

# Pre-load all summary stats files into a list and clean/standardize column names
summary_stats_data <- lapply(summary_stats_list, function(path) {
   stats <- suppressWarnings(data.table::fread(path))
   
   # Rename column if needed (uncomment if column name needs changing from "#CHROM" to "CHR")
   # setnames(stats, "#CHROM", "CHR")
   
   # Set key for efficient subsetting
   setkey(stats, SNP, CHR)
   
   # Subset to only include rows where 'CHR' matches the 'chr' variable
   stats <- stats[CHR == chr]

   # Clean and standardize column names if needed
   # clean_and_standardize_colnames(stats)
   
   return(stats)
})

summary_stats_data <- lapply(summary_stats_data, function(stats) stats[`CHR` == genome_files[g]$Chr])

print("Loaded SNP data")
print("Files for this Chr:")
print(nrow(df_this_chr))

for(j in 1:nrow(df_this_chr)){
#for(j in 1:10){
  print(paste0("File number: ", j))
  
  start_time <- Sys.time()  # Start timing the loop iteration

  if (grepl("empty", df_this_chr$path[j])) {
    message(paste0("No model for ", df_this_chr$path[j]))
    next
  }

  all_files_exist <- TRUE
  outnames <- vector("character", length(summary_stats_list))

  # Check if output files already exist
  for (k in 1:length(summary_stats_list)) {
    outnames[k] <- gsub("\\.rds$", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
    if (!file.exists(outnames[k])) {
      all_files_exist <- FALSE
      break  # Exit the loop early if any file does not exist
    }
  }

  #if (all_files_exist) {
  #  print(paste("All output files already exist for", df_this_chr$path[j], "- Skipping"))
  #  next  # Skip to the next file if all outputs already exist
  #}

  # Load the CSV file
  load_csv_start <- Sys.time()
  my_csv <- fread(df_this_chr$path[j])
  setkey(my_csv, cg)
  cgs <- levels(factor(my_csv$cg))
  #print(paste("Loaded csv file:", df_this_chr$path[j]))
  load_csv_end <- Sys.time()
  #print(paste("Time to load CSV:", load_csv_end - load_csv_start))

  # Process each summary statistics file
  for (k in 1:length(summary_stats_list)) {
    process_start <- Sys.time()
    
    outname <- gsub("_dt_combined.csv", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
    if(file.exists(outname)){
        files_skipped <- files_skipped + 1
        next
    } else {
        files_not_skipped <- files_not_skipped + 1
    }
    summary_stats <- summary_stats_data[[k]]

    MWASmodels <- vector("list", length(cgs))

    # Process each CpG site
    for (i in seq_along(cgs)) {
    #for(i in 1:10){
      this_cg <- cgs[i]
      
      process_cg_start <- Sys.time()

      this_cg_data <- my_csv[cg == this_cg]
      MWASmodels[[i]] <- process_model_csv(this_cg_data, my_SNPs, summary_stats)

      if(is.null(MWASmodels[[i]])){
        next
      }
      
      MWASmodels[[i]]['bp'] <- this_cg

      process_cg_end <- Sys.time()
  #    print(paste("Time to process CpG:", this_cg, ":", process_cg_end - process_cg_start))
    }

    # Create MWAS results
    results_start <- Sys.time()
    results <- MWASresults(MWASmodels, paths$pvar_path, paths$pgen_path, paths$psam_path, summary_stats_list[[k]], df_this_chr$path[j])
    results@MWASmodels <- Filter(Negate(is.null), results@MWASmodels)  # Remove NULL entries
    results_end <- Sys.time()
   # print(paste("Time to create MWAS results:", results_end - results_start))
    
    # Save the results
    save_start <- Sys.time()
    #saveRDS(results, outname)
    
    df <- do.call(rbind, results@MWASmodels)
    df <- as.data.frame(df)

    outname_csv <- gsub("rds", "csv", outname)
    fwrite(df, outname_csv)
    save_end <- Sys.time()
    #print(paste("Saved results to:", outname))
    #print(paste("Time to save results:", save_end - save_start))

    process_end <- Sys.time()
    #print(paste("Total time to process summary stats file", k, ":", process_end - process_start))
  }
  
  loop_end_time <- Sys.time()  # End timing the loop iteration
  print(paste("Total time for file number", j, ":", loop_end_time - start_time))
}
}

opt <- list(genome_file_index = this_chr,
            data_file = "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts/12-OUT_matched_SNP_meth_cov_outputs.csv",
            cores = 1)

#if(opt$cores > 1) registerDoParallel(opt$cores)

# Load genome files
genome_files <- list.files("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas_flipped",
                           pattern = "EUR", full.names = TRUE)
genome_files <- genome_files[grepl("allele", genome_files)]
genome_files <- genome_files[grepl("a2", genome_files)]

genome_files <- genome_files[grepl("pvar", genome_files)]

genome_files <- data.table(path = genome_files, Chr = NA)

genome_files$Chr <- str_split_fixed(genome_files$path, "chr", 2)[, 2]
genome_files$Chr <- gsub("\\.pvar", "", genome_files$Chr)

genome_files$Chr <- as.integer(genome_files$Chr)
genome_files <- genome_files[order(genome_files$Chr), ]

df <- fread(opt$data_file)
df$path <- gsub("_a2_", "_dt_", df$path)
df$path <- gsub("\\.rds", "_dt_combined.csv", df$path)

print("Starting genome file processing")
# Process the specified genome file
g <- opt$genome_file_index
print(paste("Processing genome file index:", g))

paths <- list(
  pvar_path = genome_files[g]$path,
  pgen_path = gsub("pvar", "pgen", genome_files[g]$path),
  psam_path = gsub("pvar", "psam", genome_files[g]$path)
)

chr <- as.numeric(gsub("\\.pvar", "",
            stringr::str_split_fixed(paths$pvar_path, "chr", 2)[, 2]))

my_SNPs <- CpGWAS::loadSNPData(paths$pvar_path, paths$pgen_path, paths$psam_path)
setkey(my_SNPs$pvar_dt, `#CHROM`, POS)
df_this_chr <- df[which(df$Chr == genome_files[g]$Chr), ]


summary_stats_list <- list.files("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas", pattern = "alleleprocessed", full.names = TRUE)
summary_stats_list <- summary_stats_list[grepl("a3", summary_stats_list)]

# Pre-load all summary stats files into a list and clean/standardize column names
summary_stats_data <- lapply(summary_stats_list, function(path) {
   stats <- suppressWarnings(data.table::fread(path))
   
   # Rename column if needed (uncomment if column name needs changing from "#CHROM" to "CHR")
   # setnames(stats, "#CHROM", "CHR")
   
   # Set key for efficient subsetting
   setkey(stats, SNP, CHR)
   
   # Subset to only include rows where 'CHR' matches the 'chr' variable
   stats <- stats[CHR == chr]

   # Clean and standardize column names if needed
   # clean_and_standardize_colnames(stats)
   
   return(stats)
})

summary_stats_data <- lapply(summary_stats_data, function(stats) stats[`CHR` == genome_files[g]$Chr])

print("Loaded SNP data")
print("Files for this Chr:")
print(nrow(df_this_chr))

for(j in 1:nrow(df_this_chr)){
#for(j in 1:10){
  print(paste0("File number: ", j))
  
  start_time <- Sys.time()  # Start timing the loop iteration

  if (grepl("empty", df_this_chr$path[j])) {
    message(paste0("No model for ", df_this_chr$path[j]))
    next
  }

  all_files_exist <- TRUE
  outnames <- vector("character", length(summary_stats_list))

  # Check if output files already exist
  for (k in 1:length(summary_stats_list)) {
    outnames[k] <- gsub("\\.rds$", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
    if (!file.exists(outnames[k])) {
      all_files_exist <- FALSE
      break  # Exit the loop early if any file does not exist
    }
  }

  #if (all_files_exist) {
  #  print(paste("All output files already exist for", df_this_chr$path[j], "- Skipping"))
  #  next  # Skip to the next file if all outputs already exist
  #}

  # Load the CSV file
  load_csv_start <- Sys.time()
  my_csv <- fread(df_this_chr$path[j])
  setkey(my_csv, cg)
  cgs <- levels(factor(my_csv$cg))
  #print(paste("Loaded csv file:", df_this_chr$path[j]))
  load_csv_end <- Sys.time()
  #print(paste("Time to load CSV:", load_csv_end - load_csv_start))

  # Process each summary statistics file
  for (k in 1:length(summary_stats_list)) {
    process_start <- Sys.time()
    
    outname <- gsub("_dt_combined.csv", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
    #if(file.exists(outname)) next
    summary_stats <- summary_stats_data[[k]]

    MWASmodels <- vector("list", length(cgs))

    # Process each CpG site
    for (i in seq_along(cgs)) {
    #for(i in 1:10){
      this_cg <- cgs[i]
      
      process_cg_start <- Sys.time()

      this_cg_data <- my_csv[cg == this_cg]
      MWASmodels[[i]] <- process_model_csv(this_cg_data, my_SNPs, summary_stats)

      if(is.null(MWASmodels[[i]])){
        next
      }
      
      MWASmodels[[i]]['bp'] <- this_cg

      process_cg_end <- Sys.time()
  #    print(paste("Time to process CpG:", this_cg, ":", process_cg_end - process_cg_start))
    }

    # Create MWAS results
    results_start <- Sys.time()
    results <- MWASresults(MWASmodels, paths$pvar_path, paths$pgen_path, paths$psam_path, summary_stats_list[[k]], df_this_chr$path[j])
    results@MWASmodels <- Filter(Negate(is.null), results@MWASmodels)  # Remove NULL entries
    results_end <- Sys.time()
   # print(paste("Time to create MWAS results:", results_end - results_start))
    
    # Save the results
    save_start <- Sys.time()
    #saveRDS(results, outname)
    
    df <- do.call(rbind, results@MWASmodels)
    df <- as.data.frame(df)

    outname_csv <- gsub("rds", "csv", outname)
    fwrite(df, outname_csv)
    save_end <- Sys.time()
    #print(paste("Saved results to:", outname))
    #print(paste("Time to save results:", save_end - save_start))

    process_end <- Sys.time()
    #print(paste("Total time to process summary stats file", k, ":", process_end - process_start))
  }
  
  loop_end_time <- Sys.time()  # End timing the loop iteration
  print(paste("Total time for file number", j, ":", loop_end_time - start_time))
}

Version without timing

In [ ]:
                             
# for(j in 1:nrow(df_this_chr)){
#   print(paste0("File number: ", j))
#   if (grepl("empty", df_this_chr$path[j])) {
#     message(paste0("no model for ", df_this_chr$path[j]))
#     next
#   }

#   all_files_exist <- TRUE
#   outnames <- vector("character", length(summary_stats_list))

#   for (k in 1:length(summary_stats_list)) {
#     outnames[k] <- gsub("\\.rds$", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
#     if (!file.exists(outnames[k])) {
#       all_files_exist <- FALSE
#       break  # Exit the loop early if any file does not exist
#     }
#   }

#   #if (all_files_exist) {
#    # print(paste("All output files already exist for", df_this_chr$path[j], "- Skipping"))
#    # next  # Skip to the next file if all outputs already exist
#   #}
#   #
#   my_csv <- fread(df_this_chr$path[j])
#   setkey(my_csv, cg)
#   cgs <- levels(factor(my_csv$cg))
#   print(paste("Loaded csv file:", df_this_chr$path[j]))

#   for (k in 1:length(summary_stats_list)) {
#     outname <- gsub("_dt_combined.csv", paste0("_", basename(tools::file_path_sans_ext(summary_stats_list[[k]])), "_results.rds"), df_this_chr$path[j])
#     #if(file.exists(outname)) next
#     summary_stats <- summary_stats_data[[k]]

#     MWASmodels <- vector("list", length(cgs))
# #    if (is.null(summary_stats)) {
# #      summary_stats <- suppressWarnings(fread(summary_stats_list[[k]]))
# #      summary_stats <- clean_and_standardize_colnames(summary_stats)
# #    }

#     # for (i in seq_along(my_rds@models)) {
#     #   this_MethylationBase <- my_rds@models[[i]]
#     #   if(length(this_MethylationBase@snpWeights) == 0){
#     #     next
#     #   }
#     #   MWASmodels[[i]] <- process_model(this_MethylationBase, my_SNPs, summary_stats)
#     #   MWASmodels[[i]]['bp'] <- this_MethylationBase@methylationPosition
#     #   #pb$tick()
#     # }

#     for (i in seq_along(cgs)) {
#       this_cg <- cgs[i]
      
#       this_cg_data <- my_csv[cg == this_cg]
#       MWASmodels[[i]] <- process_model_csv(this_cg_data, my_SNPs, summary_stats)

#       if(is.null(MWASmodels[[i]])){
#         next
#       }
      
#       MWASmodels[[i]]['bp'] <- MWASmodels[[i]]['bp'] <- this_cg
#       #pb$tick()
#     }

#     results <- MWASresults(MWASmodels, paths$pvar_path, paths$pgen_path, paths$psam_path, summary_stats_list[[k]], df_this_chr$path[j])

    
#     # results <- foreach(i = seq_along(my_rds@models), .packages = c("CpGWAS", "data.table"), .combine='c') %dopar% {
#     # this_MethylationBase <- my_rds@models[[i]]
#     # if (length(this_MethylationBase@snpWeights) == 0) {
#     #   return(NULL)  # Return NULL for models that should be skipped
#     # }
#     # MWASmodel <- process_model(this_MethylationBase, my_SNPs, summary_stats)
#     # MWASmodel['bp'] <- this_MethylationBase@methylationPosition
#     # return(MWASmodel)
#     # }
    
#     results@MWASmodels <- Filter(Negate(is.null), results@MWASmodels)  # Remove NULL entries
    
#     saveRDS(results, outname)

#     #filtered_models <- Filter(Negate(is.null), results@MWASmodels)
#     df <- do.call(rbind, results@MWASmodels)
#     df <- as.data.frame(df)

#     outname_csv <- gsub("rds", "csv", outname)
#     fwrite(df, outname_csv)
    
#     print(paste("Saved results to:", outname))
#   }
# }